In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.config.experimental.list_physical_devices()

In [2]:
x = tf.constant([2., 1., 0.1])
layer = layers.Softmax(axis=-1)
layer(x)

<tf.Tensor: id=2, shape=(3,), dtype=float32, numpy=array([0.6590012, 0.242433 , 0.0985659], dtype=float32)>

In [3]:
import os
os.path.abspath('.')

'd:\\gitprojects\\tensorflow_learning'

# Packaging layers

In [4]:
from tensorflow.keras import Sequential

## Method 1

In [6]:
network = Sequential([
    layers.Dense(3, activation=None),
    layers.ReLU(),
    layers.Dense(2, activation=None),
    layers.ReLU()
])

In [7]:
x = tf.random.normal([4, 3])
network(x)

<tf.Tensor: id=62, shape=(4, 2), dtype=float32, numpy=
array([[1.996155  , 0.35862863],
       [0.        , 0.04423115],
       [0.4121789 , 0.        ],
       [0.18083344, 0.        ]], dtype=float32)>

In [8]:
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  12        
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  8         
_________________________________________________________________
re_lu_1 (ReLU)               multiple                  0         
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [9]:
network.trainable_variables

[<tf.Variable 'sequential/dense/kernel:0' shape=(3, 3) dtype=float32, numpy=
 array([[-0.52368355,  0.2793069 ,  0.24653411],
        [-0.847945  ,  0.57088614, -0.8046992 ],
        [-0.86444855, -0.24619102,  0.613888  ]], dtype=float32)>,
 <tf.Variable 'sequential/dense/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'sequential/dense_1/kernel:0' shape=(3, 2) dtype=float32, numpy=
 array([[ 0.31783867, -0.74287057],
        [-0.15195787,  0.22668171],
        [ 0.9111099 ,  0.8146931 ]], dtype=float32)>,
 <tf.Variable 'sequential/dense_1/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

## Method 2

In [10]:
layers_num = 2
network = Sequential([])
for _ in range(layers_num):
    network.add(layers.Dense(3))
    network.add(layers.ReLU())

network.build(input_shape=(None, 4))
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              multiple                  15        
_________________________________________________________________
re_lu_2 (ReLU)               multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  12        
_________________________________________________________________
re_lu_3 (ReLU)               multiple                  0         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


In [11]:
for p in network.trainable_variables:
    print(p.name, p.shape)

dense_2/kernel:0 (4, 3)
dense_2/bias:0 (3,)
dense_3/kernel:0 (3, 3)
dense_3/bias:0 (3,)


# Build, train and test the model

In [5]:
from tensorflow.keras import datasets, optimizers, losses

In [6]:
(x_train, y_train),(x_test, y_test) = datasets.mnist.load_data()

In [7]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

## Build

In [15]:
network = Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10)
])
network.build(input_shape=(None, 28*28))
network.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              multiple                  200960    
_________________________________________________________________
dense_5 (Dense)              multiple                  32896     
_________________________________________________________________
dense_6 (Dense)              multiple                  8256      
_________________________________________________________________
dense_7 (Dense)              multiple                  2080      
_________________________________________________________________
dense_8 (Dense)              multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [16]:
network.compile(
    optimizer=optimizers.Adam(learning_rate=0.01),
    loss=losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

## Train

In [17]:
x_train = 2 * tf.convert_to_tensor(x_train, dtype=tf.float32)/255. -1
x_train = tf.reshape(x_train, (-1, 28*28))

y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)
y_train = tf.one_hot(y_train, depth=10)

In [18]:
train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_db = train_db.shuffle(100).batch(512)

In [19]:
x_test = 2 * tf.convert_to_tensor(x_test, dtype=tf.float32)/255. -1
x_test = tf.reshape(x_test, (-1, 28*28))

y_test = tf.convert_to_tensor(y_test, dtype=tf.int32)
y_test = tf.one_hot(y_test, depth=10)

In [20]:
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [23]:
history = network.fit(x_train,y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 8s 127us/sample - loss: 0.2765 - accuracy: 0.9302
Epoch 2/5
60000/60000 [==============================] - 8s 133us/sample - loss: 0.2861 - accuracy: 0.9294
Epoch 3/5
60000/60000 [==============================] - 9s 150us/sample - loss: 0.2788 - accuracy: 0.9302
Epoch 4/5
60000/60000 [==============================] - 8s 140us/sample - loss: 0.2889 - accuracy: 0.9292
Epoch 5/5
60000/60000 [==============================] - 8s 137us/sample - loss: 0.2665 - accuracy: 0.9351


## Test and Evaluate

In [22]:
score = network.evaluate(x_test, y_test, batch_size=128)
score

[0.2695193210788071, 0.9346]

# Save and import the model

## 1. By Tensors
### Save

In [24]:
network.save_weights('weights.ckpt')
print('saved weights')
del network

saved weights


### Import

In [15]:
network = Sequential([
    layers.Dense(256,activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10)
])
network.build(input_shape=(None, 28*28))
network.compile(
    optimizer=optimizers.Adam(learning_rate=0.01),
    loss=losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [9]:
network.load_weights('weights.ckpt')
print('loaded weights')

loaded weights


## 2. By network
### Save
Include structure and weights.

In [16]:
network.save(r'model.h5',include_optimizer=True)

In [11]:
del network

### Import

In [12]:
network = tf.keras.models.load_model('model.h5')

## 3. By SavedModel()
### Save

In [17]:
tf.keras.models.save_model(network, 'model_savedmodel', save_format='tf')
print('export saved model.')
del network

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model_savedmodel\assets
export saved model.


### Import

In [18]:
network = tf.keras.models.load_model( 'model_savedmodel')

In [19]:
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, None, 256)         200960    
_________________________________________________________________
dense_6 (Dense)              (None, None, 128)         32896     
_________________________________________________________________
dense_7 (Dense)              (None, None, 64)          8256      
_________________________________________________________________
dense_8 (Dense)              (None, None, 32)          2080      
_________________________________________________________________
dense_9 (Dense)              (None, None, 10)          330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


# Customized model

## Customize layers